# Groq AI-Powered Python Debugger

A comprehensive debugging framework that uses Groq's language models to analyze Python errors, suggest fixes, and generate detailed reports with automatic code patching capabilities.

## Table of Contents
1. [Setup and Imports](#setup)
2. [Data Classes](#data-classes)
3. [Code Analyzer](#code-analyzer)
4. [Code Patcher](#code-patcher)
5. [Main Groq Debugger](#groq-debugger)
6. [Debugging Utilities](#debugging-utilities)
7. [Usage Examples](#usage-examples)

## Setup and Imports

In [ ]:
import sys
import traceback
import ast
import inspect
import os
import json
import re
from datetime import datetime
from typing import Dict, List, Optional, Any, Callable
from dataclasses import dataclass, asdict
from groq import Groq

# Create output directory for debug reports
os.makedirs("debug_output", exist_ok=True)
print("✅ Imports loaded successfully!")

In [ ]:
# Configuration - Set your Groq API key here
GROQ_API_KEY = "your-groq-api-key-here"  # Replace with your actual API key
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

if not os.getenv("GROQ_API_KEY") or os.getenv("GROQ_API_KEY") == "your-groq-api-key-here":
    print("⚠️  Please set your GROQ_API_KEY in the cell above")
    print("   You can get one from: https://console.groq.com/keys")
else:
    print("✅ API key configured successfully!")

## Data Classes

These classes define the structure for storing debugging information, AI suggestions, and complete debugging sessions.

In [ ]:
@dataclass
class DebugInfo:
    """Container for debugging information"""
    error_type: str
    error_message: str
    traceback_info: str
    code_context: str
    line_number: int
    file_name: str
    function_name: str
    variables: Dict[str, Any]
    timestamp: str
    original_code: str

print("✅ DebugInfo class defined")

In [ ]:
@dataclass
class DebugSuggestion:
    """Container for debugging suggestions from Groq"""
    analysis: str
    suggested_fix: str
    explanation: str
    confidence: float
    alternative_solutions: List[str]
    fixed_code: str = ""
    patch_applied: bool = False

print("✅ DebugSuggestion class defined")

In [ ]:
@dataclass
class DebugSession:
    """Container for complete debugging session"""
    session_id: str
    timestamp: str
    debug_info: DebugInfo
    suggestion: DebugSuggestion
    status: str
    notes: str = ""

print("✅ DebugSession class defined")

## Code Analyzer

Analyzes Python code for potential issues and categorizes common error patterns.

In [ ]:
class CodeAnalyzer:
    """Analyzes Python code for potential issues"""

    def __init__(self):
        self.common_patterns = {
            'undefined_variable': r"name '(\w+)' is not defined",
            'indentation_error': r"IndentationError",
            'syntax_error': r"SyntaxError",
            'attribute_error': r"'(\w+)' object has no attribute '(\w+)'",
            'type_error': r"TypeError",
            'import_error': r"ModuleNotFoundError|ImportError",
            'index_error': r"IndexError",
            'key_error': r"KeyError",
            'zero_division': r"ZeroDivisionError"
        }

    def analyze_syntax(self, code: str) -> List[str]:
        """Check for syntax errors in code"""
        issues = []
        try:
            ast.parse(code)
        except SyntaxError as e:
            issues.append(f"Syntax error at line {e.lineno}: {e.msg}")
        return issues

    def get_error_pattern(self, error_message: str) -> Optional[str]:
        """Identify the pattern of the error"""
        for pattern_name, pattern in self.common_patterns.items():
            if re.search(pattern, error_message):
                return pattern_name
        return None

    def extract_original_code(self, file_path: str) -> str:
        """Extract the original code from file"""
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        except Exception:
            return "Could not read original code"

# Test the analyzer
analyzer = CodeAnalyzer()
test_code = "print('Hello World')"
issues = analyzer.analyze_syntax(test_code)
print(f"✅ CodeAnalyzer initialized. Syntax issues found: {issues}")

## Code Patcher

Applies fixes to code based on AI suggestions with pattern-specific fixing strategies.

In [ ]:
class CodePatcher:
    """Applies fixes to code based on AI suggestions"""

    def __init__(self):
        self.patch_strategies = {
            'undefined_variable': self._fix_undefined_variable,
            'zero_division': self._fix_zero_division,
            'syntax_error': self._fix_syntax_error,
            'index_error': self._fix_index_error,
            'attribute_error': self._fix_attribute_error
        }

    def apply_fix(self, original_code: str, debug_info: DebugInfo, suggestion: DebugSuggestion) -> str:
        """Apply AI suggestion to fix the code"""
        try:
            # First try to extract fixed code from AI suggestion
            fixed_code = self._extract_code_from_suggestion(suggestion.suggested_fix)
            if fixed_code and self._is_valid_python(fixed_code):
                return fixed_code

            # Fallback to pattern-based fixing
            error_pattern = CodeAnalyzer().get_error_pattern(debug_info.error_message)
            if error_pattern in self.patch_strategies:
                return self.patch_strategies[error_pattern](original_code, debug_info, suggestion)

            # Generic fix attempt
            return self._generic_fix(original_code, debug_info, suggestion)

        except Exception as e:
            return f"# Could not apply fix automatically: {str(e)}\n# Original code:\n{original_code}\n\n# AI Suggestion:\n# {suggestion.suggested_fix}"

    def _extract_code_from_suggestion(self, suggestion: str) -> Optional[str]:
        """Extract Python code from AI suggestion"""
        # Look for code blocks in markdown format
        code_blocks = re.findall(r'```python\n(.*?)\n```', suggestion, re.DOTALL)
        if code_blocks:
            return code_blocks[0].strip()

        # Look for code blocks without language specification
        code_blocks = re.findall(r'```\n(.*?)\n```', suggestion, re.DOTALL)
        if code_blocks:
            return code_blocks[0].strip()

        return None

    def _is_valid_python(self, code: str) -> bool:
        """Check if code is valid Python"""
        try:
            ast.parse(code)
            return True
        except SyntaxError:
            return False

    def _fix_undefined_variable(self, original_code: str, debug_info: DebugInfo, suggestion: DebugSuggestion) -> str:
        """Fix undefined variable errors"""
        lines = original_code.split('\n')
        error_line_idx = debug_info.line_number - 1

        if error_line_idx < len(lines):
            # Extract variable name from error
            match = re.search(r"name '(\w+)' is not defined", debug_info.error_message)
            if match:
                var_name = match.group(1)
                # Add variable definition before the error line
                indent = len(lines[error_line_idx]) - len(lines[error_line_idx].lstrip())
                new_line = ' ' * indent + f"{var_name} = None  # TODO: Define appropriate value"
                lines.insert(error_line_idx, new_line)

        return '\n'.join(lines)

    def _fix_zero_division(self, original_code: str, debug_info: DebugInfo, suggestion: DebugSuggestion) -> str:
        """Fix zero division errors"""
        lines = original_code.split('\n')
        error_line_idx = debug_info.line_number - 1

        if error_line_idx < len(lines):
            error_line = lines[error_line_idx]
            # Add zero check before division
            if '/' in error_line:
                indent = len(error_line) - len(error_line.lstrip())
                check_line = ' ' * indent + "if denominator != 0:"
                safe_line = ' ' * (indent + 4) + error_line.strip()
                else_line = ' ' * indent + "else:"
                handle_line = ' ' * (indent + 4) + "print('Error: Division by zero')"

                lines[error_line_idx] = check_line
                lines.insert(error_line_idx + 1, safe_line)
                lines.insert(error_line_idx + 2, else_line)
                lines.insert(error_line_idx + 3, handle_line)

        return '\n'.join(lines)

    def _fix_syntax_error(self, original_code: str, debug_info: DebugInfo, suggestion: DebugSuggestion) -> str:
        """Fix syntax errors"""
        lines = original_code.split('\n')
        error_line_idx = debug_info.line_number - 1

        if error_line_idx < len(lines):
            error_line = lines[error_line_idx]
            # Common syntax fixes
            if ' = ' in error_line and 'if ' in error_line:
                # Fix assignment in if statement (should be ==)
                lines[error_line_idx] = error_line.replace(' = ', ' == ')

        return '\n'.join(lines)

    def _fix_index_error(self, original_code: str, debug_info: DebugInfo, suggestion: DebugSuggestion) -> str:
        """Fix index errors"""
        lines = original_code.split('\n')
        error_line_idx = debug_info.line_number - 1

        if error_line_idx < len(lines):
            error_line = lines[error_line_idx]
            # Add bounds checking
            if '[' in error_line and ']' in error_line:
                indent = len(error_line) - len(error_line.lstrip())
                check_line = ' ' * indent + "if 0 <= index < len(sequence):"
                safe_line = ' ' * (indent + 4) + error_line.strip()
                else_line = ' ' * indent + "else:"
                handle_line = ' ' * (indent + 4) + "print('Error: Index out of range')"

                lines[error_line_idx] = check_line
                lines.insert(error_line_idx + 1, safe_line)
                lines.insert(error_line_idx + 2, else_line)
                lines.insert(error_line_idx + 3, handle_line)

        return '\n'.join(lines)

    def _fix_attribute_error(self, original_code: str, debug_info: DebugInfo, suggestion: DebugSuggestion) -> str:
        """Fix attribute errors"""
        lines = original_code.split('\n')
        error_line_idx = debug_info.line_number - 1

        if error_line_idx < len(lines):
            error_line = lines[error_line_idx]
            # Add hasattr check
            match = re.search(r"'(\w+)' object has no attribute '(\w+)'", debug_info.error_message)
            if match:
                obj_type, attr = match.groups()
                indent = len(error_line) - len(error_line.lstrip())
                check_line = ' ' * indent + f"if hasattr(obj, '{attr}'):"
                safe_line = ' ' * (indent + 4) + error_line.strip()
                else_line = ' ' * indent + "else:"
                handle_line = ' ' * (indent + 4) + f"print('Error: {attr} not found in {obj_type}')"

                lines[error_line_idx] = check_line
                lines.insert(error_line_idx + 1, safe_line)
                lines.insert(error_line_idx + 2, else_line)
                lines.insert(error_line_idx + 3, handle_line)

        return '\n'.join(lines)

    def _generic_fix(self, original_code: str, debug_info: DebugInfo, suggestion: DebugSuggestion) -> str:
        """Generic fix based on AI suggestion"""
        lines = original_code.split('\n')
        error_line_idx = debug_info.line_number - 1

        # Add comment with AI suggestion
        if error_line_idx < len(lines):
            indent = len(lines[error_line_idx]) - len(lines[error_line_idx].lstrip())
            comment_lines = [
                ' ' * indent + "# AI Suggestion:",
                ' ' * indent + f"# {suggestion.suggested_fix}",
                ' ' * indent + "# TODO: Apply the suggested fix manually"
            ]
            for i, comment in enumerate(comment_lines):
                lines.insert(error_line_idx + i, comment)

        return '\n'.join(lines)

# Test the patcher
patcher = CodePatcher()
print("✅ CodePatcher initialized successfully!")

## Main Groq Debugger

The core debugging framework that integrates with Groq's AI models for intelligent error analysis and fixing.

In [ ]:
class GroqDebugger:
    """Main debugging framework using Groq models with JSON output"""

    def __init__(self, api_key: str, model: str = "meta-llama/llama-4-maverick-17b-128e-instruct", output_dir: str = "debug_output"):
        self.client = Groq(api_key=api_key)
        self.model = model
        self.analyzer = CodeAnalyzer()
        self.patcher = CodePatcher()
        self.debug_history: List[DebugSession] = []
        self.output_dir = output_dir
        self.session_counter = 0

        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)

    def capture_error_context(self, exc_type, exc_value, exc_traceback) -> DebugInfo:
        """Capture detailed error context"""
        tb = traceback.extract_tb(exc_traceback)
        last_frame = tb[-1]

        # Get local variables from the frame
        frame = exc_traceback.tb_frame
        variables = {}
        try:
            variables = {k: repr(v) for k, v in frame.f_locals.items()
                        if not k.startswith('__') and len(repr(v)) < 1000}
        except Exception:
            variables = {"error": "Could not capture variables"}

        # Get code context
        try:
            with open(last_frame.filename, 'r', encoding='utf-8') as f:
                lines = f.readlines()
                start = max(0, last_frame.lineno - 5)
                end = min(len(lines), last_frame.lineno + 5)
                context_lines = lines[start:end]
                code_context = ''.join(f"{start + i + 1:4d}: {line}"
                                     for i, line in enumerate(context_lines))
        except Exception:
            code_context = "Could not retrieve code context"

        # Get original code
        original_code = self.analyzer.extract_original_code(last_frame.filename)

        return DebugInfo(
            error_type=exc_type.__name__,
            error_message=str(exc_value),
            traceback_info=''.join(traceback.format_exception(exc_type, exc_value, exc_traceback)),
            code_context=code_context,
            line_number=last_frame.lineno,
            file_name=last_frame.filename,
            function_name=last_frame.name,
            variables=variables,
            timestamp=datetime.now().isoformat(),
            original_code=original_code
        )

    def format_debug_prompt(self, debug_info: DebugInfo) -> str:
        """Format debugging information for Groq model"""
        prompt = f"""
You are an expert Python debugger. Analyze the following error and provide detailed debugging assistance with actual fixed code.

ERROR INFORMATION:
- Error Type: {debug_info.error_type}
- Error Message: {debug_info.error_message}
- File: {debug_info.file_name}
- Function: {debug_info.function_name}
- Line: {debug_info.line_number}

CODE CONTEXT:
{debug_info.code_context}

ORIGINAL CODE:
```python
{debug_info.original_code}
```

LOCAL VARIABLES:
{json.dumps(debug_info.variables, indent=2)}

Please provide:
1. A clear analysis of what caused the error
2. Complete fixed code that resolves the issue
3. An explanation of why this error occurred and how the fix works
4. A confidence score (0.0 to 1.0) for your suggestion
5. Alternative solutions if applicable

Format your response as JSON with the following structure:
{{
    "analysis": "detailed analysis of the error",
    "suggested_fix": "```python\\n[complete fixed code here]\\n```",
    "explanation": "why this error happened and how the fix works",
    "confidence": 0.95,
    "alternative_solutions": ["solution 1", "solution 2"]
}}

Make sure the suggested_fix contains complete, working Python code that can be executed without errors.
"""
        return prompt

print("✅ GroqDebugger class (Part 1) loaded successfully!")

In [ ]:
# Continue GroqDebugger class - Part 2

    def get_debug_suggestion(self, debug_info: DebugInfo) -> DebugSuggestion:
        """Get debugging suggestion from Groq model"""
        prompt = self.format_debug_prompt(debug_info)

        try:
            response = self.client.chat.completions.create(
                messages=[
                    {"role": "user", "content": prompt}
                ],
                model=self.model,
                temperature=0.1,
                max_tokens=3000
            )

            content = response.choices[0].message.content

            # Try to parse JSON response
            try:
                suggestion_data = json.loads(content)
                suggestion = DebugSuggestion(**suggestion_data)

                # Apply the fix to get actual fixed code
                suggestion.fixed_code = self.patcher.apply_fix(
                    debug_info.original_code, debug_info, suggestion
                )
                suggestion.patch_applied = True

                return suggestion

            except json.JSONDecodeError:
                # Fallback if JSON parsing fails
                fixed_code = self.patcher.apply_fix(debug_info.original_code, debug_info,
                                                  DebugSuggestion(analysis=content, suggested_fix=content,
                                                                explanation="AI provided analysis", confidence=0.5,
                                                                alternative_solutions=[]))
                return DebugSuggestion(
                    analysis=content,
                    suggested_fix=content,
                    explanation="AI provided detailed analysis",
                    confidence=0.5,
                    alternative_solutions=[],
                    fixed_code=fixed_code,
                    patch_applied=True
                )

        except Exception as e:
            return DebugSuggestion(
                analysis=f"Error communicating with Groq: {str(e)}",
                suggested_fix="Manual debugging required",
                explanation="Could not get AI assistance",
                confidence=0.0,
                alternative_solutions=[],
                fixed_code=debug_info.original_code,
                patch_applied=False
            )

# Add the methods to the existing GroqDebugger class
GroqDebugger.get_debug_suggestion = get_debug_suggestion

print("✅ GroqDebugger class (Part 2) methods added successfully!")

In [ ]:
# Continue GroqDebugger class - Part 3 (File handling methods)

def save_debug_session_to_json(self, debug_session: DebugSession) -> str:
    """Save debug session to JSON file"""
    filename = f"debug_session_{debug_session.session_id}.json"
    filepath = os.path.join(self.output_dir, filename)

    # Convert dataclass to dictionary for JSON serialization
    session_dict = {
        "session_info": {
            "session_id": debug_session.session_id,
            "timestamp": debug_session.timestamp,
            "status": debug_session.status,
            "notes": debug_session.notes
        },
        "error_details": asdict(debug_session.debug_info),
        "ai_analysis": {
            "analysis": debug_session.suggestion.analysis,
            "suggested_fix": debug_session.suggestion.suggested_fix,
            "explanation": debug_session.suggestion.explanation,
            "confidence": debug_session.suggestion.confidence,
            "alternative_solutions": debug_session.suggestion.alternative_solutions,
            "patch_applied": debug_session.suggestion.patch_applied
        },
        "code_changes": {
            "original_code": debug_session.debug_info.original_code,
            "fixed_code": debug_session.suggestion.fixed_code,
            "has_fix": bool(debug_session.suggestion.fixed_code.strip())
        }
    }

    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(session_dict, f, indent=2, ensure_ascii=False)
        return filepath
    except Exception as e:
        print(f"Error saving debug session: {e}")
        return ""

def save_summary_report(self) -> str:
    """Save summary report of all debugging sessions"""
    summary_file = os.path.join(self.output_dir, "debug_summary.json")

    summary = {
        "report_generated": datetime.now().isoformat(),
        "total_sessions": len(self.debug_history),
        "sessions": []
    }

    for session in self.debug_history:
        session_summary = {
            "session_id": session.session_id,
            "timestamp": session.timestamp,
            "error_type": session.debug_info.error_type,
            "error_message": session.debug_info.error_message,
            "file_name": session.debug_info.file_name,
            "line_number": session.debug_info.line_number,
            "confidence": session.suggestion.confidence,
            "patch_applied": session.suggestion.patch_applied,
            "status": session.status
        }
        summary["sessions"].append(session_summary)

    try:
        with open(summary_file, 'w', encoding='utf-8') as f:
            json.dump(summary, f, indent=2, ensure_ascii=False)
        return summary_file
    except Exception as e:
        print(f"Error saving summary report: {e}")
        return ""

# Add the methods to the existing GroqDebugger class
GroqDebugger.save_debug_session_to_json = save_debug_session_to_json
GroqDebugger.save_summary_report = save_summary_report

print("✅ GroqDebugger class (Part 3) file handling methods added successfully!")

In [ ]:
# Continue GroqDebugger class - Part 4 (Main debugging methods)

def debug_with_json_output(self, debug_info: DebugInfo) -> str:
    """Main debugging method with JSON output"""
    self.session_counter += 1
    session_id = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_{self.session_counter:03d}"

    print(f"\n🐛 Starting Debug Session: {session_id}")
    print(f"📍 Error: {debug_info.error_type} at {debug_info.file_name}:{debug_info.line_number}")

    # Get AI suggestion
    print("🤖 Getting AI analysis...")
    suggestion = self.get_debug_suggestion(debug_info)

    # Create debug session
    debug_session = DebugSession(
        session_id=session_id,
        timestamp=datetime.now().isoformat(),
        debug_info=debug_info,
        suggestion=suggestion,
        status="completed" if suggestion.patch_applied else "manual_review_required",
        notes=f"AI confidence: {suggestion.confidence:.2%}"
    )

    # Save to history
    self.debug_history.append(debug_session)

    # Save to JSON file
    json_file = self.save_debug_session_to_json(debug_session)

    if json_file:
        print(f"💾 Debug session saved to: {json_file}")
        print(f"📊 AI Confidence: {suggestion.confidence:.2%}")
        if suggestion.patch_applied:
            print("✅ Code fix applied successfully")
        else:
            print("⚠️  Manual review required for code fix")

    # Update summary report
    summary_file = self.save_summary_report()
    if summary_file:
        print(f"📋 Summary report updated: {summary_file}")

    return json_file

def debug_code_static(self, code: str, filename: str = "static_analysis.py") -> str:
    """Analyze code for potential issues without execution"""
    issues = self.analyzer.analyze_syntax(code)

    # Create mock debug info for static analysis
    debug_info = DebugInfo(
        error_type="StaticAnalysis",
        error_message="Code review and potential issue detection",
        traceback_info="No runtime error - static analysis",
        code_context=code,
        line_number=1,
        file_name=filename,
        function_name="static_analysis",
        variables={},
        timestamp=datetime.now().isoformat(),
        original_code=code
    )

    if not issues:
        # Use Groq to analyze for logical issues
        prompt = f"""
Analyze this Python code for potential runtime errors, logic issues, or improvements:

```python
{code}
```

Provide specific feedback as JSON:
{{
    "analysis": "detailed analysis of potential issues",
    "suggested_fix": "```python\\n[improved code]\\n```",
    "explanation": "explanation of issues and improvements",
    "confidence": 0.8,
    "alternative_solutions": ["improvement 1", "improvement 2"]
}}
"""

        try:
            response = self.client.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model=self.model,
                temperature=0.1,
                max_tokens=2000
            )

            content = response.choices[0].message.content
            try:
                suggestion_data = json.loads(content)
                suggestion = DebugSuggestion(**suggestion_data)
                suggestion.fixed_code = self.patcher.apply_fix(code, debug_info, suggestion)
                suggestion.patch_applied = True
            except json.JSONDecodeError:
                suggestion = DebugSuggestion(
                    analysis=content,
                    suggested_fix=content,
                    explanation="Static analysis completed",
                    confidence=0.7,
                    alternative_solutions=[],
                    fixed_code=code,
                    patch_applied=False
                )

        except Exception as e:
            suggestion = DebugSuggestion(
                analysis=f"Could not complete static analysis: {str(e)}",
                suggested_fix="Manual review required",
                explanation="Static analysis failed",
                confidence=0.0,
                alternative_solutions=[],
                fixed_code=code,
                patch_applied=False
            )

    else:
        # Handle syntax errors
        suggestion = DebugSuggestion(
            analysis=f"Syntax issues found: {'; '.join(issues)}",
            suggested_fix="Fix syntax errors manually",
            explanation="Code has syntax errors that prevent execution",
            confidence=0.9,
            alternative_solutions=["Use IDE syntax checker", "Review Python syntax"],
            fixed_code=code,
            patch_applied=False
        )

    return self.debug_with_json_output(debug_info)

# Add the methods to the existing GroqDebugger class
GroqDebugger.debug_with_json_output = debug_with_json_output
GroqDebugger.debug_code_static = debug_code_static

print("✅ GroqDebugger class (Part 4) main debugging methods added successfully!")

## Debugging Utilities

Convenience classes for automatic debugging using decorators and context managers.

In [ ]:
class DebugDecorator:
    """Decorator for automatic debugging with JSON output"""

    def __init__(self, debugger: GroqDebugger):
        self.debugger = debugger

    def __call__(self, func: Callable) -> Callable:
        def wrapper(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except Exception:
                exc_type, exc_value, exc_traceback = sys.exc_info()
                debug_info = self.debugger.capture_error_context(
                    exc_type, exc_value, exc_traceback
                )
                self.debugger.debug_with_json_output(debug_info)
                raise
        return wrapper


class DebugContext:
    """Context manager for debugging code blocks with JSON output"""

    def __init__(self, debugger: GroqDebugger):
        self.debugger = debugger

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, exc_traceback):
        if exc_type is not None:
            debug_info = self.debugger.capture_error_context(
                exc_type, exc_value, exc_traceback
            )
            self.debugger.debug_with_json_output(debug_info)
        return False  # Don't suppress the exception

print("✅ Debugging utilities loaded successfully!")

## Usage Examples

Practical examples demonstrating how to use the debugging framework.

### Initialize the Debugger

In [ ]:
# Initialize debugger (make sure to set your API key above first!)
if os.getenv("GROQ_API_KEY") and os.getenv("GROQ_API_KEY") != "your-groq-api-key-here":
    debugger = GroqDebugger(os.getenv("GROQ_API_KEY"), output_dir="debug_output")
    print("✅ Debugger initialized successfully!")
    print(f"📁 Debug reports will be saved to: {debugger.output_dir}")
else:
    print("❌ Please set your GROQ_API_KEY in the configuration cell above")
    print("   You can get one from: https://console.groq.com/keys")

### Example 1: Static Code Analysis

In [ ]:
# Analyze code without running it
problematic_code = """
def divide_numbers(a, b):
    result = a / b  # Potential division by zero
    return result

def process_list(items):
    return items[10]  # Potential index error

# Missing variable definition
print(undefined_variable)
"""

if 'debugger' in locals():
    print("🔍 Analyzing code for potential issues...")
    json_file = debugger.debug_code_static(problematic_code, "example_analysis.py")
    print(f"📄 Analysis saved to: {json_file}")
else:
    print("❌ Debugger not initialized. Please run the initialization cell first.")

### Example 2: Using Debug Decorator

In [ ]:
# Example using the debug decorator
if 'debugger' in locals():
    debug_decorator = DebugDecorator(debugger)
    
    @debug_decorator
    def buggy_function(x, y):
        """Function with intentional bug for demonstration"""
        if x > 0:
            return y / x  # Will fail if x is 0
        else:
            return undefined_var  # NameError
    
    print("🧪 Testing decorated function...")
    try:
        result = buggy_function(0, 10)  # This will trigger the debugger
    except Exception as e:
        print(f"Exception caught: {e}")
else:
    print("❌ Debugger not initialized. Please run the initialization cell first.")

### Example 3: Using Debug Context Manager

In [ ]:
# Example using context manager
if 'debugger' in locals():
    debug_context = DebugContext(debugger)
    
    print("🧪 Testing context manager...")
    try:
        with debug_context:
            # Code block that might have errors
            numbers = [1, 2, 3]
            print(numbers[10])  # IndexError
    except Exception as e:
        print(f"Exception caught: {e}")
else:
    print("❌ Debugger not initialized. Please run the initialization cell first.")

### Example 4: View Debug History

In [ ]:
# View debugging session history
if 'debugger' in locals():
    print("📊 Debug Session History:")
    print(f"Total sessions: {len(debugger.debug_history)}")
    
    for i, session in enumerate(debugger.debug_history, 1):
        print(f"\n{i}. Session {session.session_id}")
        print(f"   Error: {session.debug_info.error_type}")
        print(f"   Confidence: {session.suggestion.confidence:.2%}")
        print(f"   Status: {session.status}")
        print(f"   Time: {session.timestamp}")
else:
    print("❌ Debugger not initialized or no debug sessions yet.")

### Example 5: Examine Debug Output Files

In [ ]:
# List and examine debug output files
import os
import json
from pprint import pprint

debug_dir = "debug_output"
if os.path.exists(debug_dir):
    files = [f for f in os.listdir(debug_dir) if f.endswith('.json')]
    print(f"📁 Found {len(files)} debug files:")
    
    for file in files:
        print(f"   • {file}")
    
    # Show contents of the latest debug session
    if files and 'debug_session_' in str(files):
        session_files = [f for f in files if f.startswith('debug_session_')]
        if session_files:
            latest_file = sorted(session_files)[-1]
            filepath = os.path.join(debug_dir, latest_file)
            
            print(f"\n📄 Latest debug session ({latest_file}):")
            with open(filepath, 'r') as f:
                data = json.load(f)
                print(f"   Error Type: {data['error_details']['error_type']}")
                print(f"   Confidence: {data['ai_analysis']['confidence']:.2%}")
                print(f"   Status: {data['session_info']['status']}")
                
                # Show a snippet of the AI analysis
                if 'analysis' in data['ai_analysis']:
                    analysis = data['ai_analysis']['analysis']
                    print(f"   AI Analysis: {analysis[:100]}..." if len(analysis) > 100 else f"   AI Analysis: {analysis}")
else:
    print("❌ Debug output directory not found.")

### Example 6: Test with a Simple Bug

In [ ]:
# Simple test to demonstrate the debugger in action
if 'debugger' in locals():
    print("🧪 Testing with a simple division by zero bug...")
    
    simple_buggy_code = """
def divide_by_zero():
    x = 10
    y = 0
    result = x / y  # This will cause a ZeroDivisionError
    return result

print(divide_by_zero())
"""
    
    json_file = debugger.debug_code_static(simple_buggy_code, "simple_bug_test.py")
    print(f"📄 Analysis completed and saved to: {json_file}")
else:
    print("❌ Debugger not initialized. Please run the initialization cell first.")

## Features Summary

**🎯 Key Features:**
- **AI-Powered Analysis**: Uses Groq's language models for intelligent error analysis
- **Automatic Code Fixing**: Applies suggested fixes with pattern-based strategies  
- **Comprehensive Logging**: Saves detailed JSON reports for each debugging session
- **Multiple Usage Patterns**: Decorators, context managers, and direct analysis
- **Static Code Analysis**: Analyze code without execution
- **Error Pattern Recognition**: Identifies common error types and applies specific fixes

**🛠️ Supported Error Types:**
- Undefined variables
- Zero division errors  
- Syntax errors
- Index errors
- Attribute errors
- Import errors
- Type errors
- And more...

**📊 Output Formats:**
- Detailed JSON reports for each session
- Summary reports across all sessions
- Fixed code with applied patches
- Confidence scores and alternative solutions

## Next Steps

1. **Set your Groq API key** in the configuration cell
2. **Run the cells** in order to set up the framework
3. **Try the examples** to see the debugger in action
4. **Use in your own code** by applying decorators or context managers
5. **Examine the JSON outputs** to understand the AI's analysis and suggestions

The framework provides a powerful way to automatically analyze, understand, and fix Python code errors using AI assistance!